In [1]:
%matplotlib notebook
%pwd

'/ocean/projects/asc170022p/mtragoza/lung-project'

In [2]:
import sys, os, re
import yaml
from pathlib import Path
import numpy as np
import xarray as xr
import hvplot.xarray

In [3]:
!tree --sort=mtime -L 1 data/4DCT

data/4DCT
├── Case1Pack
├── Case2Pack
├── Case3Pack
├── Case4Pack
├── Case5Pack
├── Case7Pack
├── Case8Pack
├── Case9Pack
├── Case10Pack
└── Case6Pack

10 directories, 0 files


In [4]:
!tree --sort=mtime data/4DCT/Case1Pack

data/4DCT/Case1Pack
├── Sampled4D
│   ├── case1_4D-75_T00.txt
│   ├── case1_4D-75_T50.txt
│   ├── case1_4D-75_T10.txt
│   ├── case1_4D-75_T20.txt
│   ├── case1_4D-75_T30.txt
│   └── case1_4D-75_T40.txt
├── ExtremePhases
│   ├── Case1_300_T00_xyz.txt
│   └── Case1_300_T50_xyz.txt
├── Images
│   ├── case1_T00_s.img
│   ├── case1_T10_s.img
│   ├── case1_T20_s.img
│   ├── case1_T30_s.img
│   ├── case1_T40_s.img
│   ├── case1_T50_s.img
│   ├── case1_T60_s.img
│   ├── case1_T70_s.img
│   ├── case1_T80_s.img
│   └── case1_T90_s.img
└── case1.yaml

3 directories, 19 files


In [5]:
def read_xyz_file(xyz_file):
    '''
    Read landmark xyz coordinates from text file.
    '''
    with open(xyz_file) as f:
        data = [line.strip().split() for line in f]
    return np.array(data, dtype=np.uint8)

xyz = read_xyz_file('data/4DCT/Case1Pack/Sampled4D/case1_4D-75_T00.txt')
xyz.shape

(75, 3)

In [6]:
def load_img_file(img_file, shape, dtype, verbose=True):
    '''
    Read CT image from file in Analyze 7.5 format.
    
    https://stackoverflow.com/questions/27507928/loading-analyze-7-5-format-images-in-python
    '''
    if verbose:
        print(f'Loading {img_file}')
    data = np.fromfile(img_file, dtype)
    data = data.reshape(shape)
    itemsize = data.dtype.itemsize
    data.strides = (
        itemsize,
        itemsize * shape[0],
        itemsize * shape[0] * shape[1]
    )
    return data.copy()

image = load_img_file('data/4DCT/Case1Pack/Images/case1_T00_s.img', shape=(256, 256, 94), dtype=np.int16)
image.shape

Loading data/4DCT/Case1Pack/Images/case1_T00_s.img


(256, 256, 94)

In [7]:
def load_yaml_file(yaml_file):
    print(f'Loading {yaml_file}')
    with open(yaml_file) as f:
        return yaml.safe_load(f)
    
load_yaml_file('data/4DCT/Case1Pack/case1.yaml')

Loading data/4DCT/Case1Pack/case1.yaml


{'shape': [256, 256, 94], 'resolution': [0.97, 0.97, 2.5]}

In [61]:
class Lung4DCTPatient(object):
    
    def __init__(self, data_root, case_id, phase):
        self.data_root = Path(data_root)
        self.case_id = int(case_id)
        self.phase = list(phase)
        
    @property
    def patient_dir(self):
        return self.data_root / f'Case{self.case_id}Pack'
    
    @property
    def metadata_file(self):
        return self.patient_dir / f'case{self.case_id}.yaml'
    
    @property
    def image_dir(self):
        return self.patient_dir / 'Images'

    def load_metadata(self):
        metadata = load_yaml_file(self.metadata_file)
        self.shape = metadata['shape']
        self.resolution = metadata['resolution']
        
    def load_images(self):

        images = []
        for phase in self.phase:
            img_glob = self.image_dir.glob(f'case{self.case_id}_T{phase:02d}*.img')
            img_file = next(img_glob) # assumes exactly one match
            image = load_img_file(img_file, self.shape, dtype=np.int16)
            images.append(image)
        
        self.array = xr.DataArray(
            data=np.stack(images)[...,::-1], # flip z orientation
            dims=['t', 'x', 'y', 'z'],
            coords={
                't': self.phase,
                'x': np.arange(self.shape[0]) * self.resolution[0],
                'y': np.arange(self.shape[1]) * self.resolution[1],
                'z': np.arange(self.shape[2]) * self.resolution[2]
            },
            name='anatomy'
        )
        
    def view(self, select=None, **kwargs):
        q1, q2, q3 = self.array.quantile([0.25, 0.50, 0.75])
        median, iqr = q2, (q3 - q1)
        default_kws = {
            'cmap': 'greys_r',
            'clim': (0, median + 1.5 * iqr),
            'data_aspect': 1,
            'width': 500
        }
        default_kws.update(**kwargs)
        if select:
            return self.array.sel(**select).hvplot(**default_kws)
        else:
            return self.array.hvplot(**default_kws)

p = Lung4DCTPatient(data_root='data/4DCT', case_id=1, phase=range(0, 100, 10))
p.load_metadata()
p.load_images()
p.array

Loading data/4DCT/Case1Pack/case1.yaml
Loading data/4DCT/Case1Pack/Images/case1_T00_s.img
Loading data/4DCT/Case1Pack/Images/case1_T10_s.img
Loading data/4DCT/Case1Pack/Images/case1_T20_s.img
Loading data/4DCT/Case1Pack/Images/case1_T30_s.img
Loading data/4DCT/Case1Pack/Images/case1_T40_s.img
Loading data/4DCT/Case1Pack/Images/case1_T50_s.img
Loading data/4DCT/Case1Pack/Images/case1_T60_s.img
Loading data/4DCT/Case1Pack/Images/case1_T70_s.img
Loading data/4DCT/Case1Pack/Images/case1_T80_s.img
Loading data/4DCT/Case1Pack/Images/case1_T90_s.img


<xarray.DataArray 'anatomy' (t: 10, x: 256, y: 256, z: 94)>
array([[[[ 37,  37,  40, ...,  34,  28,  16],
         [ 37,  26,  47, ...,  23,  21,  17],
         [ 46,  31,  20, ...,  21,  33,  20],
         ...,
         [580, 584, 603, ..., 284, 360, 305],
         [308, 298, 324, ..., 153, 137, 169],
         [185, 177, 188, ..., 115, 103, 125]],

        [[ 38,  32,  23, ...,  36,  25,  12],
         [ 25,  31,  36, ...,  26,  26,  13],
         [ 23,  34,  46, ...,  21,  22,  24],
         ...,
         [591, 587, 615, ..., 300, 356, 297],
         [316, 297, 311, ..., 166, 148, 152],
         [190, 184, 184, ..., 125, 111, 115]],

        [[ 28,  33,  29, ...,  35,   9,  31],
         [ 33,  35,  16, ...,  34,  26,  21],
         [ 29,  27,  44, ...,  41,  23,  15],
         ...,
...
         ...,
         [311, 308, 303, ..., 154, 221, 159],
         [157, 182, 165, ..., 115, 135, 121],
         [151, 143, 146, ..., 109, 132, 100]],

        [[ 33,  42,  20, ...,  30,  22,  25],
         [ 28,  47,  34, ...,  33,  20,  28],
         [ 34,  30,  36, ...,  23,  20,  16],
         ...,
         [324, 309, 306, ..., 157, 225, 160],
         [157, 175, 164, ..., 133, 141, 113],
         [140, 152, 133, ..., 109, 127,  95]],

        [[ 32,  29,  27, ...,  19,  29,  32],
         [ 31,  22,  41, ...,  21,  24,  25],
         [ 46,  24,  38, ...,  24,  23,  28],
         ...,
         [300, 311, 309, ..., 151, 222, 161],
         [168, 170, 155, ..., 134, 124, 134],
         [142, 147, 142, ..., 116, 101, 113]]]], dtype=int16)
Coordinates:
  * t        (t) int64 0 10 20 30 40 50 60 70 80 90
  * x        (x) float64 0.0 0.97 1.94 2.91 3.88 ... 244.4 245.4 246.4 247.3
  * y        (y) float64 0.0 0.97 1.94 2.91 3.88 ... 244.4 245.4 246.4 247.3
  * z        (z) float64 0.0 2.5 5.0 7.5 10.0 ... 222.5 225.0 227.5 230.0 232.5

In [62]:
p.view(groupby=['t', 'y'], x='x', y='z')

/ocean/projects/asc170022p/mtragoza/mambaforge/envs/4DCT/lib/python3.11/site-packages/holoviews/core/util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)


:DynamicMap   [t,y]
   :Image   [x,z]   (anatomy)

In [72]:
p.view(select={'y': 126.1}, groupby='t', x='x', y='z', widget_type='scrubber', widget_location='bottom')

/ocean/projects/asc170022p/mtragoza/mambaforge/envs/4DCT/lib/python3.11/site-packages/holoviews/core/util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)


Column
    [0] HoloViews(DynamicMap, sizing_mode='fixed', widget_location='bottom', widget_type='scrubber')
    [1] WidgetBox(align=('center', 'end'))
        [0] Player(end=9, width=550)